**Importing Liabraries**

In [1]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.layers import InputSpec,Layer
# from keras.engine import InputSpec, Layer
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping
import os
import pandas as pd
import urduhack
urduhack.download()
from urduhack.models.lemmatizer import lemmatizer
from urduhack.normalization import normalize
from urduhack.preprocessing import normalize_whitespace, remove_punctuation,remove_accents,replace_numbers
from urduhack.stop_words import STOP_WORDS
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Custom Function for Stopwords Removal
def remove_stopwords(text: str):
    return " ".join(word for word in text.split() if word not in STOP_WORDS)

In [3]:
# Custom Function For Applying Lemma
def lemitizeStr(str):
    lemme_str = ""
    temp = lemmatizer.lemma_lookup(str)
    for t in temp:
        lemme_str += t[0] + " "
    
    return lemme_str

# Preprocessing With Urdu Hack

**Text Preprocessing of Orignal Document**

In [4]:
import pandas as pd 



In [5]:
original_file_read = pd.read_xml('./UPPC Corpus/data/001_task_a.xml',xpath="/UPPC_document")
original_file_data = original_file_read['UPPC_document'].values[0]

In [6]:
print(original_file_data)
print(len(original_file_data))

چودھری رحمت علی  16 نومبر1897  کو مشرقی پنجاب کے ضلع ہوشیار پور کے گائوں  موہراں میں ایک
 متوسط زمیندار جناب حاجی شاہ کے ہاں پیدا   ہوئے۔ ابتدائی تعلیم انہوں نے ایک مکتب سے حاصل کی جو ایک عالم دین چلا رہے تھے۔ میٹرک اینگلو سنسکرت ہائی اسکول جالندھر سے کیا۔ 1914  میں   مزید تعلیم کے لئے لاہور  تشریف  لائے انہوں نے اسلامیہ کالج لاہور میں داخلہ لیا۔ 1915 میں اسلامیہ کالج میں بزم  شبلی کی بنیاد رکھی کیونکہ وہ  مولاناشبلی سے بہت  متاثر تھے اور پھراس کے پلیٹ فارم  سے 1915 میں تقسیم  ہن کا نظریہ  پیش کیا۔ 1918 میں بی اے کرنے کے بعد جناب محمد  دین فوق کے اخبار کشمیر گزٹ میں اسسٹنٹ ایڈیٹر کی حیثیت سے اپنے کیئریر کا آغاز کیا۔ 1928 میں ایچی سن کالج میں اتالیق مقرر ہوئے۔ کچھ  عرصہ  بعد انگلستان تشریف لے گئے جہاں کیمبرج اور ڈبلن یونورسٹیوں سے قانون اور سیاست میں اعلیٰ ڈگریاں حاصل کیں۔ اس طرح 1933 میں انہوں نے برصغیر کے طلباء پر مشتمل ایک تنظیم پاکستان نیشنل لبریشن موومنٹ کے نام سے قائم کی۔ ذہن میں رکھئے گا 1933 میں۔اسی سال چودھری رحمت علی  نے دوسری گول می کانفرنس کے موقع  پر اپنا مشہور کتابچہ Now o

In [7]:
# original_file_data = replace_numbers(original_file_data)
original_file_data = normalize(original_file_data)
original_file_data = remove_accents(original_file_data)
original_file_data = remove_punctuation(original_file_data)
original_file_data = normalize_whitespace(original_file_data)

In [8]:
print(original_file_data)
print(len(original_file_data))

چودھری رحمت علی 16 نومبر1897 کو مشرقی پنجاب کے ضلع ہوشیار پور کے گائوں موہراں میں ایک
 متوسط زمیندار جناب حاجی شاہ کے ہاں پیدا ہوئے ابتدائی تعلیم انہوں نے ایک مکتب سے حاصل کی جو ایک عالم دین چلا رہے تھے میٹرک اینگلو سنسکرت ہائی اسکول جالندھر سے کیا 1914 میں مزید تعلیم کے لئے لاہور تشریف لائے انہوں نے اسلامیہ کالج لاہور میں داخلہ لیا 1915 میں اسلامیہ کالج میں بزم شبلی کی بنیاد رکھی کیونکہ وہ مولاناشبلی سے بہت متاثر تھے اور پھراس کے پلیٹ فارم سے 1915 میں تقسیم ہن کا نظریہ پیش کیا 1918 میں بی اے کرنے کے بعد جناب محمد دین فوق کے اخبار کشمیر گزٹ میں اسسٹنٹ ایڈیٹر کی حیثیت سے اپنے کیئریر کا آغاز کیا 1928 میں ایچی سن کالج میں اتالیق مقرر ہوئے کچھ عرصہ بعد انگلستان تشریف لے گئے جہاں کیمبرج اور ڈبلن یونورسٹیوں سے قانون اور سیاست میں اعلی ڈگریاں حاصل کیں اس طرح 1933 میں انہوں نے برصغیر کے طلباء پر مشتمل ایک تنظیم پاکستان نیشنل لبریشن موومنٹ کے نام سے قائم کی ذہن میں رکھئے گا 1933 میںاسی سال چودھری رحمت علی نے دوسری گول می کانفرنس کے موقع پر اپنا مشہور کتابچہ Now or Never اب یا کبھی نہیں شائع کیا

In [9]:
original_file_data = remove_stopwords(original_file_data)

In [10]:
print(original_file_data)
print(len(original_file_data))

چودھری رحمت علی 16 نومبر1897 مشرقی پنجاب ضلع ہوشیار پور گائوں موہراں متوسط زمیندار حاجی شاہ پیدا ابتدائی تعلیم مکتب حاصل عالم دین میٹرک اینگلو سنسکرت ہائی اسکول جالندھر 1914 تعلیم لاہور تشریف اسلامیہ کالج لاہور داخلہ 1915 اسلامیہ کالج بزم شبلی بنیاد مولاناشبلی متاثر پھراس پلیٹ فارم 1915 تقسیم ہن نظریہ پیش 1918 بی اے محمد دین فوق اخبار کشمیر گزٹ اسسٹنٹ ایڈیٹر حیثیت کیئریر آغاز 1928 ایچی سن کالج اتالیق مقرر عرصہ انگلستان تشریف کیمبرج ڈبلن یونورسٹیوں قانون سیاست اعلی ڈگریاں حاصل کیں 1933 برصغیر طلباء مشتمل تنظیم پاکستان نیشنل لبریشن موومنٹ نام قائم ذہن رکھئے 1933 میںاسی سال چودھری رحمت علی گول می کانفرنس موقع مشہور کتابچہ Now or Never شائع مرتبہ لفظ پاکستان استعمال 1935 کیمبرج ہفت روزہ اخبار نکالا نام پاکستان چودھری رحمت علی 23 مارچ آلانڈیا مسلم لیگ چونتیسوی سالانہ اجلاس لاہور تشریف روز خاکساروں فائرنگ وقت وزیر اعلی پنجاب سکندر حیات چودھری رحمت علی پنجاب داخلے پابندی عائد دی29 جنوری 1951 نمونیہ مبتلا شدید بیماری حالت انگلستان مقامی نرسنگ ہوم داخل صحت یاب سکے 3 فروری 1951 خالق حقیقی ملی چو

In [11]:
lemmatized = lemitizeStr(original_file_data)

In [12]:
lemmatized

'چودھری رحمت علی 16 نومبر1897 مشرقی پنجاب ضلع ہوشیار پور گائوں موہراں متوسط زمیندار حاجی شاہ پیدا ابتدائی تعلیم مکتب حاصل عالم دین میٹرک اینگلو سنسکرت ہائی اسکول جالندھر 1914 تعلیم لاہور تشریف اسلامیہ کالج لاہور داخلہ 1915 اسلامیہ کالج بزم شبلی بنیاد مولاناشبلی متاثر پھراس پلیٹ فارم 1915 تقسیم ہن نظریہ پیش 1918 بی اے محمد دین فوق اخبار کشمیر گزٹ اسسٹنٹ ایڈیٹر حیثیت کیئریر آغاز 1928 ایچی سن کالج اتالیق مقرر عرصہ انگلستان تشریف کیمبرج ڈبلن یونورسٹیوں قانون سیاست اعلی ڈگریاں حاصل کیں 1933 برصغیر طلباء مشتمل تنظیم پاکستان نیشنل لبریشن موومنٹ نام قائم ذہن رکھئے 1933 میںاسی سال چودھری رحمت علی گول می کانفرنس موقع مشہور کتابچہ Now or Never شائع مرتبہ لفظ پاکستان استعمال 1935 کیمبرج ہفت روزہ اخبار نکالا نام پاکستان چودھری رحمت علی 23 مارچ آلانڈیا مسلم لیگ چونتیسوی سالانہ اجلاس لاہور تشریف روز خاکساروں فائرنگ وقت وزیر اعلی پنجاب سکندر حیات چودھری رحمت علی پنجاب داخلے پابندی عائد دی29 جنوری 1951 نمونیہ مبتلا شدید بیماری حالت انگلستان مقامی نرسنگ ہوم داخل صحت یاب سکے 3 فروری 1951 خالق حقیقی ملی چ

In [13]:
len(original_file_data)

1060

In [14]:
len(lemmatized)

1061

In [15]:
tokenizer_original = spacy.blank('ur')

tokenized_original = tokenizer_original(lemmatized)

In [16]:
tokenized_original

چودھری رحمت علی 16 نومبر1897 مشرقی پنجاب ضلع ہوشیار پور گائوں موہراں متوسط زمیندار حاجی شاہ پیدا ابتدائی تعلیم مکتب حاصل عالم دین میٹرک اینگلو سنسکرت ہائی اسکول جالندھر 1914 تعلیم لاہور تشریف اسلامیہ کالج لاہور داخلہ 1915 اسلامیہ کالج بزم شبلی بنیاد مولاناشبلی متاثر پھراس پلیٹ فارم 1915 تقسیم ہن نظریہ پیش 1918 بی اے محمد دین فوق اخبار کشمیر گزٹ اسسٹنٹ ایڈیٹر حیثیت کیئریر آغاز 1928 ایچی سن کالج اتالیق مقرر عرصہ انگلستان تشریف کیمبرج ڈبلن یونورسٹیوں قانون سیاست اعلی ڈگریاں حاصل کیں 1933 برصغیر طلباء مشتمل تنظیم پاکستان نیشنل لبریشن موومنٹ نام قائم ذہن رکھئے 1933 میںاسی سال چودھری رحمت علی گول می کانفرنس موقع مشہور کتابچہ Now or Never شائع مرتبہ لفظ پاکستان استعمال 1935 کیمبرج ہفت روزہ اخبار نکالا نام پاکستان چودھری رحمت علی 23 مارچ آلانڈیا مسلم لیگ چونتیسوی سالانہ اجلاس لاہور تشریف روز خاکساروں فائرنگ وقت وزیر اعلی پنجاب سکندر حیات چودھری رحمت علی پنجاب داخلے پابندی عائد دی29 جنوری 1951 نمونیہ مبتلا شدید بیماری حالت انگلستان مقامی نرسنگ ہوم داخل صحت یاب سکے 3 فروری 1951 خالق حقیقی ملی چو

In [56]:

glove_embeddings = {}
with open("glove.840B.300d.txt",encoding="utf8") as f:

    for line in f:

        try:
            line = line.split()
            glove_embeddings[line[0]] = np.array(line[1:], dtype=np.float32)
        except:
            continue

In [58]:
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences

max_tokens = 50 ## Hyperparameter

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(tokenized_original)

## Vectorizing data to keep 50 words per sample.
X_train_vect = pad_sequences(tokenizer.texts_to_sequences(tokenized_original), maxlen=max_tokens, padding="post", truncating="post", value=0.)
# X_test_vect  = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=max_tokens, padding="post", truncating="post", value=0.)

print(X_train_vect[:3])

X_train_vect.shape

TypeError: 'int' object is not callable

In [ ]:
embedding_path = "D:\\NLP Task\\glove.840B.300d.txt"
embed_size = 300
max_features = 30000

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

In [19]:
# doc = TfidfVectorizer(preprocessor=callable).fit_transform(tokenized_original)

**Text Preprocessing of NP Document**

In [20]:
pnp_file_read = pd.read_xml('./UPPC Corpus/data/086_task_a_np.xml',xpath="/UPPC_document")
pnp_file_data = pnp_file_read['UPPC_document'].values[0]

XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)

In [21]:
print(pnp_file_data)
print(len(pnp_file_data))

NameError: name 'pnp_file_data' is not defined

In [ ]:
pnp_file_data = replace_numbers(pnp_file_data)
pnp_file_data = normalize(pnp_file_data)
pnp_file_data = remove_accents(pnp_file_data)
pnp_file_data = remove_punctuation(pnp_file_data)
pnp_file_data = normalize_whitespace(pnp_file_data)

In [22]:
print(pnp_file_data)
print(len(pnp_file_data))

NameError: name 'pnp_file_data' is not defined

In [23]:
pnp_file_data = remove_stopwords(pnp_file_data)

NameError: name 'pnp_file_data' is not defined

In [24]:
print(pnp_file_data)
print(len(pnp_file_data))

NameError: name 'pnp_file_data' is not defined

In [ ]:
lemmatized = lemitizeStr(pnp_file_data)

In [25]:
lemmatized

'چودھری رحمت علی 16 نومبر1897 مشرقی پنجاب ضلع ہوشیار پور گائوں موہراں متوسط زمیندار حاجی شاہ پیدا ابتدائی تعلیم مکتب حاصل عالم دین میٹرک اینگلو سنسکرت ہائی اسکول جالندھر 1914 تعلیم لاہور تشریف اسلامیہ کالج لاہور داخلہ 1915 اسلامیہ کالج بزم شبلی بنیاد مولاناشبلی متاثر پھراس پلیٹ فارم 1915 تقسیم ہن نظریہ پیش 1918 بی اے محمد دین فوق اخبار کشمیر گزٹ اسسٹنٹ ایڈیٹر حیثیت کیئریر آغاز 1928 ایچی سن کالج اتالیق مقرر عرصہ انگلستان تشریف کیمبرج ڈبلن یونورسٹیوں قانون سیاست اعلی ڈگریاں حاصل کیں 1933 برصغیر طلباء مشتمل تنظیم پاکستان نیشنل لبریشن موومنٹ نام قائم ذہن رکھئے 1933 میںاسی سال چودھری رحمت علی گول می کانفرنس موقع مشہور کتابچہ Now or Never شائع مرتبہ لفظ پاکستان استعمال 1935 کیمبرج ہفت روزہ اخبار نکالا نام پاکستان چودھری رحمت علی 23 مارچ آلانڈیا مسلم لیگ چونتیسوی سالانہ اجلاس لاہور تشریف روز خاکساروں فائرنگ وقت وزیر اعلی پنجاب سکندر حیات چودھری رحمت علی پنجاب داخلے پابندی عائد دی29 جنوری 1951 نمونیہ مبتلا شدید بیماری حالت انگلستان مقامی نرسنگ ہوم داخل صحت یاب سکے 3 فروری 1951 خالق حقیقی ملی چ

In [26]:
len(pnp_file_data)

NameError: name 'pnp_file_data' is not defined

In [27]:
tokenizer_pnp = spacy.blank('ur')

tokenized_pnp = tokenizer_pnp(pnp_file_data)

NameError: name 'pnp_file_data' is not defined

In [ ]:
tokenized_pnp[0]

چودھری

In [31]:
embedding_path = "D:\\NLP Task\\glove.840B.300d.txt"
embed_size = 300
max_features = 30000

def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 962: character maps to <undefined>

In [21]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

train_dir = os.path.join('D:\\NLP Task\\UPPC Corpus\\data\\Train')
train_dataset = tf.keras.utils_dataset_from_directory(
    train_dir, label_mode='int', labels='inferred', follow_links = True
)
test_dir = os.path.join('D:\\NLP Task\\UPPC Corpus\\data\\Test')
test_dataset = tf.keras.utils.text_dataset_from_directory(
    test_dir, label_mode='int', labels='inferred', follow_links = True
)

VOCAB_SIZE = 5000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))
# Building the Recurrent Neural Network
# using GRU cells and Hyperbolic tangent as activation function
cell = tf.keras.layers.GRUCell(30, recurrent_activation='tanh')
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.RNN(cell)),
    tf.keras.layers.Dense(60, activation='tanh'),
    tf.keras.layers.Dense(1)
])
# Compile model and use the algorithm Adam as optimization function
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(1e-2), metrics=['accuracy'])
# Fitting the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=10)
# Model Evaluation
test_loss, test_acc = model.evaluate(test_dataset)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)
# Visualize Model Loss and Accuracy
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

Found 0 files belonging to 0 classes.


ValueError: No text files found in directory D:\NLP Task\UPPC Corpus\data\Train. Allowed format: .txt